<a href="https://colab.research.google.com/github/msabelhaus/geo-practice/blob/main/fast_food_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Following this guide: https://medium.com/walmartglobaltech/introduction-to-spatial-analytics-in-python-44e92b4af362

In [28]:
# Imports
import pandas as pd
import geopandas as gpd
import folium
from pyproj import CRS
from shapely.geometry import Point
import numpy as np
# from geopy.distance import geodesi

In [21]:
#create_point - combines lat/long to a single point
#Function to create a point based on the latitude and longitude columns in a dataframe. Returns original dataframe with point geometry as a column
def create_point (df, Long, Lat):
    crs = CRS("epsg:4326") # uses the Coordinate Reference System (CRS) to convert into geospatial data
    geometry = [Point(xy) for xy in zip(df[Long], df[Lat])] # Since we have just lat/long, the spatial data is a point
    df = gpd.GeoDataFrame(df, geometry=geometry).set_crs(crs, allow_override=True)
    df.rename(columns={'geometry': 'Centroid'}, inplace=True)
    return df

#create_buffer_area - takes a list of radii and creates a buffer area based on points in a dataframe
#Need to specify the geometry column in the dataframe when defining the funciton
#When crs is set to epsg:2272, this is commonly used for the measure of feet, which can be easily converted into miles (5280ft in a mile)
#Good reference for crs is https://epsg.io
def create_buffer_area (df, geometry_col, radius_list):
    gdf = gpd.GeoDataFrame(df, geometry=geometry_col) # Convert df into geodataframe
    gdf.to_crs(epsg=2272, inplace=True)
    rdf = gpd.GeoDataFrame() # Initialize empty dataframe to store result
    for r in radius_list:
      # For each radius in radius_list, create buffer around each geometry in gdf
        tadf = pd.merge(df,
                       gpd.GeoDataFrame(gdf.geometry.buffer(r*5280)), # Multiply by 5280 to convert miles to feet
                       left_index=True,
                       right_index=True)
        tadf['Radius'], tadf['Radius_Type'] = r, 'Mile'
        rdf = pd.concat([rdf,tadf])
    rdf.rename(columns={0: 'Buffer_Area'}, inplace=True)
    rdf = gpd.GeoDataFrame(rdf, geometry=geometry_col)
    print(rdf.head())
    # rdf.to_crs(epsg=4326, inplace=True)
    return rdf

#plot_point_polygon - plots a latitude, longitudes, and trade area from a dataframe onto a map
def plot_point_polygon (df, latitude, longitude, trade_area):
    #Create base map and centers the Map
    middle_long = np.median([df[longitude]])
    middle_lat = np.median([df[latitude]])
    m = folium.Map(location=[middle_lat, middle_long], width='80%', height='80%', tiles='OpenStreetMap', zoom_start=7)
    #m = folium.Map(location=[39.828395, -98.579480], zoom_start=5)
    #Plot Latitude & Longitude
    for idx, r in df.iterrows():
        folium.Marker([r[latitude], r[longitude]]).add_to(m)
    #Plot polygons
    polys = df[trade_area]
    folium.GeoJson(polys.to_crs(epsg=4326)).add_to(m)
    return m

#setgeometry - quickly create a geometry column and set the crs
def setgeometry(df, geometry_variable, crs_type):
    df['geometry'] = df[geometry_variable]
    df = df.set_geometry("geometry")
    df.to_crs(epsg=crs_type, inplace=True)
    return df

#spatial_match will find points and/or radii that fall within or overlap with another radius
def spatial_match(df1,geometry1_col,df2,geometry2_col,crs_type=2272,how_type='inner'):
    df1 = setgeometry(df1,geometry1_col,crs_type)
    df2 = setgeometry(df2,geometry2_col,crs_type)
    new_df = gpd.sjoin(df1,df2,how=how_type, op='intersects')
    return new_df

In [31]:
# Load dataset
url = 'https://raw.githubusercontent.com/msabelhaus/geo-practice/main/data/FastFoodRestaurants.csv'
df_FFR = pd.read_csv(url)
df_FFR = df_FFR[['name','address','city','province','latitude','longitude']] # subset to columns of interest
df_FFR.rename(columns={'province':'state'}, inplace=True) # rename provine as state

## Creating Data Subsets
# Mcdonalds in NY
df_MD_CO=df_FFR.loc[(df_FFR.state=='NY')&(df_FFR.name.isin(['McDonald\'s','Mcdonald\'s','McDonalds']))]
df_MD_CO.reset_index(drop=True, inplace=True)
# Burger King in NY
df_BK_CO = df_FFR.loc[(df_FFR.state=='NY') & (df_FFR.name.isin(['Burger King']))]
df_BK_CO.reset_index(drop=True, inplace=True)

## Creating Data Subsets
# Mcdonalds in CO
df_MD_CO=df_FFR.loc[(df_FFR.state=='CO')&(df_FFR.name.isin(['McDonald\'s','Mcdonald\'s','McDonalds']))]
df_MD_CO.reset_index(drop=True, inplace=True)
# Burger King in CO
df_BK_CO = df_FFR.loc[(df_FFR.state=='CO') & (df_FFR.name.isin(['Burger King']))]
df_BK_CO.reset_index(drop=True, inplace=True)

In [32]:
df_BK_CO.head()

,name,address,city,state,latitude,longitude
0,Burger King,295 Eby Creek,Eagle,CO,39.706980,-106.840280
1,Burger King,1895 S Nevada Ave,Colorado Springs,CO,38.806945,-104.822204
2,Burger King,1727 E Platte Ave,Colorado Springs,CO,38.836800,-104.794900
3,Burger King,4875 Federal Blvd,Denver,CO,39.785300,-105.025700
4,Burger King,375 S Academy Blvd,Colorado Springs,CO,38.825865,-104.757365


In [33]:
# Create Centroid Point from Lat/Long for both McDonalds and Burger Kings
df_MD_CO = create_point(df_MD_CO, 'longitude', 'latitude')
df_BK_CO = create_point(df_BK_CO, 'longitude', 'latitude')

In [34]:
df_MD_CO.head()

,name,address,city,state,latitude,longitude,Centroid
0,McDonald's,2171 N Frontage Rd W,Vail,CO,39.629750,-106.41908,POINT (-106.41908 39.62975)
1,McDonald's,10011 Jordan Rd Unit A,Parker,CO,39.534313,-104.79436,POINT (-104.79436 39.53431)
2,McDonald's,524 E Us Highway 24,Woodland Park,CO,38.989720,-105.04762,POINT (-105.04762 38.98972)
3,Mcdonald's,3112 S Glen Ave,Glenwood Springs,CO,39.519500,-107.32003,POINT (-107.32003 39.51950)
4,McDonald's,390 S 8th St,Colorado Springs,CO,38.828500,-104.84130,POINT (-104.84130 38.82850)


In [35]:
# Create a 3 mile buffer area polygon around each McDonalds
radii = [3]  # list of trade area radius. Here it's only a 3 mile radius
df_MD_CO = create_buffer_area(df_MD_CO, 'Centroid', radii) # The new geometry is a polygon around each point

         name                 address              city state   latitude  \
0  McDonald's    2171 N Frontage Rd W              Vail    CO  39.629750   
1  McDonald's  10011 Jordan Rd Unit A            Parker    CO  39.534313   
2  McDonald's     524 E Us Highway 24     Woodland Park    CO  38.989720   
3  Mcdonald's         3112 S Glen Ave  Glenwood Springs    CO  39.519500   
4  McDonald's            390 S 8th St  Colorado Springs    CO  38.828500   

   longitude                          Centroid  \
0 -106.41908  POINT (-5966120.047 1407347.253)   
1 -104.79436  POINT (-5541524.777 1232218.258)   
2 -105.04762  POINT (-5669963.922 1064858.749)   
3 -107.32003  POINT (-6219431.374 1451589.455)   
4 -104.84130   POINT (-5631798.612 991082.878)   

                                         Buffer_Area  Radius Radius_Type  
0  POLYGON ((-5950280.047 1407347.253, -5950356.3...       3        Mile  
1  POLYGON ((-5525684.777 1232218.258, -5525761.0...       3        Mile  
2  POLYGON ((-565

# Visualization

In [36]:
# Creating map
ffr_map = plot_point_polygon(df_MD_CO, 'latitude', 'longitude', 'Buffer_Area')
ffr_map

# Spatial join

In [37]:
# Renaming columns before spatial join to avoid ambiguity
df_MD_CO.columns = ['MD_' + col for col in df_MD_CO.columns]
df_BK_CO.columns = ['BK_' + col for col in df_BK_CO.columns]

In [38]:
# Use spatial_match to find the Burger Kings that are located within 3 miles of McDonald’s in New York.
BK_near_MD_CO = spatial_match(df_MD_CO,'MD_Buffer_Area',df_BK_CO,'BK_Centroid')
BK_near_MD_CO = BK_near_MD_CO[['MD_latitude','MD_longitude','MD_Centroid','MD_Buffer_Area','BK_latitude','BK_longitude','BK_Centroid']]
BK_near_MD_CO.reset_index(drop=True,inplace=True)
# Marking the intersecting Burger King points ar red points in the map
for idx, r in BK_near_MD_CO.iterrows():
        folium.CircleMarker([r['BK_latitude'], r['BK_longitude']],radius=2,color='red').add_to(ffr_map)
# visualizing the map
ffr_map

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
